In [12]:
import numpy as np
import pandas as pd
import random

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

import methods


In [13]:
n = 10
columns = ['N', 'late_penality', 'waiting_charge', 'actual_time']
for i in range(1, n + 1):
    columns.append(f'a{i}')
df = pd.DataFrame(columns=columns)

In [14]:
for _ in range(10000):
    N = random.randint(n + 11, n + 39)
    intervals = np.random.gamma(shape=4, scale=2, size=N+n)

    # Variable gamma
    intervals = np.random.gamma(shape=random.randint(3, 7), scale=2, size=N+n)
    
    first_n = intervals[:n]
    late_penalty = random.randint(25, 500)
    waiting_charge = random.uniform(0.5, 1.8)
    actual_time = methods.cal_actual_time(n=5, intervals=intervals)
    dic = {'N': N, 'late_penality': late_penalty, 'waiting_charge': waiting_charge, 'actual_time': actual_time}
    for i in range(1, n + 1):
        dic[f'a{i}'] = first_n[i - 1]
    df.loc[len(df)] = dic
    # df.append(dic, ignore_index=True)


In [15]:
df.head()


,N,late_penality,waiting_charge,actual_time,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10
0,22,121,1.404333,228.250069,12.593619,16.932170,10.076747,7.666508,8.211519,13.138036,9.945555,4.157682,5.215360,19.312088
1,46,313,0.675931,413.118777,11.974191,3.561050,5.707289,14.162098,12.593269,7.229602,11.646953,9.562969,7.291728,7.682174
2,32,204,1.799425,161.637836,4.356968,5.751433,3.040969,10.097363,4.399642,13.182752,2.239995,8.550151,3.722218,2.794255
3,32,352,1.203262,392.363832,19.054359,19.013230,6.400656,16.476727,18.864876,11.992971,7.483035,7.224410,12.865871,9.808800
4,33,400,0.935263,251.198640,9.323367,8.851960,7.313354,7.019534,2.113108,7.425334,10.799419,3.518908,5.094155,6.789569


In [16]:
features = [x for x in columns if x != 'actual_time']
X = df[features]
y = df['actual_time']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
model = LinearRegression()

model.fit(X_train, y_train)
print(model.score(X_train, y_train))
model.coef_

0.8654469595313583


array([ 9.90978921e+00, -3.95021202e-03,  2.19625995e+00,  1.83888333e+00,
        2.00710904e+00,  2.04816928e+00,  2.00450026e+00,  2.01630624e+00,
        3.01603148e+00,  3.00580080e+00,  2.78439174e+00,  3.01343531e+00,
        3.20259856e+00])

In [18]:
print(model.score(X_test, y_test))

0.8703346810340128


In [19]:
gamma_shape = 4
gamma_scale = 2
N = 40
late_penalty = 100
waiting_charge = 1
costs = []
delays = []
for _ in range(10000):
    intervals = np.random.gamma(shape=4, scale=2, size=N+n)
    first_n = intervals[:n]
    dic = {'N': [N], 'late_penality': [late_penalty], 'waiting_charge': [waiting_charge]}
    for i in range(1, n + 1):
        dic[f'a{i}'] = [first_n[i - 1]]
    predicted_time = model.predict(pd.DataFrame(dic))[0] - 10
    actual_time = methods.cal_actual_time(n=n, intervals=intervals)
    cost = methods.cal_cost(c=late_penalty, h=waiting_charge, actual_time=actual_time, predicted_time=predicted_time)
    costs.append(cost)
    delays.append(actual_time - predicted_time)
methods.plot_plotly(data=costs, data_label='Cost')
methods.plot_plotly(data=delays, data_label='Delay')